In [3]:
!pip install nltk

  Created wheel for nltk: filename=nltk-3.5-py3-none-any.whl size=1434677 sha256=80291f970d03aa6a80226b6373236346cbf42e9a549b63abfe1ad81cab7a9a36
  Stored in directory: c:\users\georges\appdata\local\pip\cache\wheels\45\6c\46\a1865e7ba706b3817f5d1b2ff7ce8996aabdd0d03d47ba0266
Successfully built nltk


In [3]:
import nltk
import re
import numpy as np

## Citirea textului din fisier

In [4]:
with open("input.txt", "r") as fd:
    input_text = fd.read()

 ## Preprocesarea textului

#### Impartirea pe propozitii

In [17]:
import nltk
nltk.download('punkt')
from nltk.corpus import stopwords

sentences = nltk.sent_tokenize(input_text)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\GeorgeS\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [6]:
sentences

['Lorem Ipsum is simply dummy text of the printing and typesetting industry.',
 "Lorem Ipsum has been the industry's standard dummy text ever since the 1500s, when an unknown printer took a galley of type and scrambled it to make a type specimen book.",
 'It has survived not only five centuries, but also the leap into electronic typesetting, remaining essentially unchanged.',
 'It was popularised in the 1960s with the release of Letraset sheets containing Lorem Ipsum passages, and more recently with desktop publishing software like Aldus PageMaker including versions of Lorem Ipsum']

#### Tokenizarea si eliminarea stopwords-urilor

In [20]:
import re
word_set = set()
s = []
for i in sentences:
    sentence = re.sub("[^a-zA-Z]", " ",  i.lower())
    y = [x for x in nltk.word_tokenize(sentence) if x not in stopwords.words('english')]
    s.append(y)
    word_set.update(y)
words_in_sentences = s

In [23]:
words_in_sentences

[['lorem',
  'ipsum',
  'simply',
  'dummy',
  'text',
  'printing',
  'typesetting',
  'industry'],
 ['lorem',
  'ipsum',
  'industry',
  'standard',
  'dummy',
  'text',
  'ever',
  'since',
  'unknown',
  'printer',
  'took',
  'galley',
  'type',
  'scrambled',
  'make',
  'type',
  'specimen',
  'book'],
 ['survived',
  'five',
  'centuries',
  'also',
  'leap',
  'electronic',
  'typesetting',
  'remaining',
  'essentially',
  'unchanged'],
 ['popularised',
  'release',
  'letraset',
  'sheets',
  'containing',
  'lorem',
  'ipsum',
  'passages',
  'recently',
  'desktop',
  'publishing',
  'software',
  'like',
  'aldus',
  'pagemaker',
  'including',
  'versions',
  'lorem',
  'ipsum']]

In [21]:
word_set

{'aldus',
 'also',
 'book',
 'centuries',
 'containing',
 'desktop',
 'dummy',
 'electronic',
 'essentially',
 'ever',
 'five',
 'galley',
 'including',
 'industry',
 'ipsum',
 'leap',
 'letraset',
 'like',
 'lorem',
 'make',
 'pagemaker',
 'passages',
 'popularised',
 'printer',
 'printing',
 'publishing',
 'recently',
 'release',
 'remaining',
 'scrambled',
 'sheets',
 'simply',
 'since',
 'software',
 'specimen',
 'standard',
 'survived',
 'text',
 'took',
 'type',
 'typesetting',
 'unchanged',
 'unknown',
 'versions'}

## Generarea datelor de antrenare: vectori one-hot

In [25]:
import numpy as np
word_set = list(word_set)
word_indices = np.array([x for x in range(len(word_set))])

one_hot_vectors = np.zeros((word_indices.size, word_indices.max() + 1))
one_hot_vectors[np.arange(word_indices.size),word_indices] = 1

In [26]:
one_hot_vectors

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])